# Fine Tuning dei Modelli Open AI

Questo notebook si basa sulle indicazioni attuali fornite nella documentazione [Fine Tuning](https://platform.openai.com/docs/guides/fine-tuning?WT.mc_id=academic-105485-koreyst) di Open AI.

Il fine-tuning migliora le prestazioni dei modelli di base per la tua applicazione riaddestrandoli con dati aggiuntivi e contesti rilevanti per quel caso d'uso o scenario specifico. Nota che tecniche di prompt engineering come il _few shot learning_ e la _retrieval augmented generation_ ti permettono di migliorare il prompt predefinito con dati rilevanti per aumentare la qualità. Tuttavia, questi approcci sono limitati dalla dimensione massima della finestra di token del modello di base target.

Con il fine-tuning, stiamo effettivamente riaddestrando il modello stesso con i dati richiesti (consentendoci di usare molti più esempi di quanti ne possano entrare nella finestra massima di token) - e distribuendo una versione _personalizzata_ del modello che non necessita più di avere esempi forniti al momento dell'inferenza. Questo non solo migliora l'efficacia del design del nostro prompt (abbiamo più flessibilità nell'usare la finestra di token per altre cose) ma potenzialmente migliora anche i nostri costi (riducendo il numero di token che dobbiamo inviare al modello al momento dell'inferenza).

Il fine tuning ha 4 passaggi:
1. Preparare i dati di addestramento e caricarli.
1. Eseguire il job di addestramento per ottenere un modello fine-tuned.
1. Valutare il modello fine-tuned e iterare per la qualità.
1. Distribuire il modello fine-tuned per l'inferenza quando soddisfatti.

Nota che non tutti i modelli di base supportano il fine-tuning - [controlla la documentazione OpenAI](https://platform.openai.com/docs/guides/fine-tuning/what-models-can-be-fine-tuned?WT.mc_id=academic-105485-koreyst) per le informazioni più aggiornate. Puoi anche fare il fine-tuning di un modello precedentemente fine-tuned. In questo tutorial, useremo `gpt-35-turbo` come modello di base target per il fine-tuning.

---


### Passo 1.1: Prepara il tuo dataset

Costruiamo un chatbot che ti aiuti a comprendere la tavola periodica degli elementi rispondendo a domande su un elemento con un limerick. In _questo_ semplice tutorial, creeremo solo un dataset per addestrare il modello con alcuni esempi di risposte che mostrano il formato previsto dei dati. In un caso d'uso reale, dovresti creare un dataset con molti più esempi. Potresti anche essere in grado di utilizzare un dataset aperto (per il tuo dominio applicativo) se ne esiste uno, e riformattarlo per l'uso nel fine-tuning.

Poiché ci stiamo concentrando su `gpt-35-turbo` e cerchiamo una risposta a turno singolo (completamento chat), possiamo creare esempi usando [questo formato suggerito](https://platform.openai.com/docs/guides/fine-tuning/preparing-your-dataset?WT.mc_id=academic-105485-koreyst) che riflette i requisiti del completamento chat di OpenAI. Se ti aspetti contenuti conversazionali a più turni, useresti il [formato di esempio multi-turno](https://platform.openai.com/docs/guides/fine-tuning/multi-turn-chat-examples?WT.mc_id=academic-105485-koreyst) che include un parametro `weight` per segnalare quali messaggi dovrebbero essere usati (o meno) nel processo di fine-tuning.

Useremo il formato più semplice a turno singolo per il nostro tutorial qui. I dati sono nel formato [jsonl](https://jsonlines.org/?WT.mc_id=academic-105485-koreyst) con 1 record per riga, ciascuno rappresentato come un oggetto formattato JSON. Lo snippet qui sotto mostra 2 record come esempio - vedi [training-data.jsonl](../../../../../18-fine-tuning/python/openai/training-data.jsonl) per il set completo di esempi (10 esempi) che useremo per il nostro tutorial di fine-tuning. **Nota:** Ogni record _deve_ essere definito in una singola riga (non diviso su più righe come tipico in un file JSON formattato)

```json
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Gallium"}, {"role": "assistant", "content": "Gallium, oh gallium, so light - Melts in your hand, oh what a sight - At 86 degrees - Its liquid with ease - And in semiconductors, it's out of sight"}]}
{ "messages": [{"role": "system", "content": "Elle is a factual chatbot that answers questions about elements in the periodic table with a limerick"}, {"role": "user", "content": "Tell me about Hydrogen"}, {"role": "assistant", "content": "Hydrogen, the first in the line - The lightest of all, so divine - It's in water, you see - And in stars, it's the key - The universe's most common sign"}]}
```

In un caso d'uso reale avrai bisogno di un set di esempi molto più ampio per ottenere buoni risultati - il compromesso sarà tra la qualità delle risposte e il tempo/costi per il fine-tuning. Stiamo usando un set piccolo così da poter completare rapidamente il fine-tuning per illustrare il processo. Vedi [questo esempio del Cookbook OpenAI](https://github.com/openai/openai-cookbook/blob/main/examples/How_to_finetune_chat_models.ipynb?WT.mc_id=academic-105485-koreyst) per un tutorial di fine-tuning più complesso.


---

### Passo 1.2 Carica il tuo Dataset

Carica i dati utilizzando l'API Files [come descritto qui](https://platform.openai.com/docs/guides/fine-tuning/upload-a-training-file). Nota che per eseguire questo codice, devi aver prima completato i seguenti passaggi:
 - Installato il pacchetto Python `openai` (assicurati di usare una versione >=0.28.0 per le funzionalità più recenti)
 - Impostato la variabile d'ambiente `OPENAI_API_KEY` con la tua chiave API di OpenAI
Per saperne di più, consulta la [Guida all'installazione](./../../../00-course-setup/02-setup-local.md?WT.mc_id=academic-105485-koreyst) fornita per il corso.

Ora, esegui il codice per creare un file da caricare dal tuo file JSONL locale.


In [24]:
from openai import OpenAI
client = OpenAI()

ft_file = client.files.create(
  file=open("./training-data.jsonl", "rb"),
  purpose="fine-tune"
)

print(ft_file)
print("Training File ID: " + ft_file.id)

FileObject(id='file-JdAJcagdOTG6ACNlFWzuzmyV', bytes=4021, created_at=1715566183, filename='training-data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)
Training File ID: file-JdAJcagdOTG6ACNlFWzuzmyV


---

### Passo 2.1: Crea il lavoro di Fine-tuning con l’SDK


In [25]:
from openai import OpenAI
client = OpenAI()

ft_filejob = client.fine_tuning.jobs.create(
  training_file=ft_file.id, 
  model="gpt-3.5-turbo"
)

print(ft_filejob)
print("Fine-tuning Job ID: " + ft_filejob.id)

FineTuningJob(id='ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', created_at=1715566184, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-EZ6ag0n0S6Zm8eV9BSWKmE6l', result_files=[], seed=830529052, status='validating_files', trained_tokens=None, training_file='file-JdAJcagdOTG6ACNlFWzuzmyV', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)
Fine-tuning Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh


---

### Passo 2.2: Controlla lo stato del lavoro

Ecco alcune cose che puoi fare con l'API `client.fine_tuning.jobs`:
- `client.fine_tuning.jobs.list(limit=<n>)` - Elenca gli ultimi n lavori di fine-tuning
- `client.fine_tuning.jobs.retrieve(<job_id>)` - Ottieni i dettagli di un lavoro di fine-tuning specifico
- `client.fine_tuning.jobs.cancel(<job_id>)` - Annulla un lavoro di fine-tuning
- `client.fine_tuning.jobs.list_events(fine_tuning_job_id=<job_id>, limit=<b>)` - Elenca fino a n eventi dal lavoro
- `client.fine_tuning.jobs.create(model="gpt-35-turbo", training_file="your-training-file.jsonl", ...)`

Il primo passo del processo è _validare il file di addestramento_ per assicurarsi che i dati siano nel formato corretto.


In [26]:
from openai import OpenAI
client = OpenAI()

# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve(ft_filejob.id)

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id=ft_filejob.id, limit=10)

SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-GkWiDgZmOsuv4q5cSTEGscY6', created_at=1715566184, level='info', message='Validating training file: file-JdAJcagdOTG6ACNlFWzuzmyV', object='fine_tuning.job.event', data={}, type='message'), FineTuningJobEvent(id='ftevent-3899xdVTO3LN7Q7LkKLMJUnb', created_at=1715566184, level='info', message='Created fine-tuning job: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh', object='fine_tuning.job.event', data={}, type='message')], object='list', has_more=False)

In [30]:
# Once the training data is validated
# Track the job status to see if it is running and when it is complete
from openai import OpenAI
client = OpenAI()

response = client.fine_tuning.jobs.retrieve(ft_filejob.id)

print("Job ID:", response.id)
print("Status:", response.status)
print("Trained Tokens:", response.trained_tokens)

Job ID: ftjob-Usfb9RjasncaZ5Cjbuh1XSCh
Status: running
Trained Tokens: None


---

### Passo 2.3: Tracciare gli eventi per monitorare i progressi


In [44]:
# You can also track progress in a more granular way by checking for events
# Refresh this code till you get the `The job has successfully completed` message
response = client.fine_tuning.jobs.list_events(ft_filejob.id)

events = response.data
events.reverse()

for event in events:
    print(event.message)

Step 85/100: training loss=0.14
Step 86/100: training loss=0.00
Step 87/100: training loss=0.00
Step 88/100: training loss=0.07
Step 89/100: training loss=0.00
Step 90/100: training loss=0.00
Step 91/100: training loss=0.00
Step 92/100: training loss=0.00
Step 93/100: training loss=0.00
Step 94/100: training loss=0.00
Step 95/100: training loss=0.08
Step 96/100: training loss=0.05
Step 97/100: training loss=0.00
Step 98/100: training loss=0.00
Step 99/100: training loss=0.00
Step 100/100: training loss=0.00
Checkpoint created at step 80 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyyF2:ckpt-step-80
Checkpoint created at step 90 with Snapshot ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWyzhK:ckpt-step-90
New fine-tuned model created: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz
The job has successfully completed


### Passo 2.4: Visualizza lo stato nella Dashboard di OpenAI


Puoi anche visualizzare lo stato visitando il sito web di OpenAI ed esplorando la sezione _Fine-tuning_ della piattaforma. Questo ti mostrerà lo stato del lavoro corrente e ti permetterà anche di tracciare la cronologia delle esecuzioni dei lavori precedenti. In questo screenshot, puoi vedere che l'esecuzione precedente è fallita e la seconda esecuzione è riuscita. Per contesto, questo è successo quando la prima esecuzione ha utilizzato un file JSON con record formattati in modo errato - una volta corretto, la seconda esecuzione è stata completata con successo e ha reso il modello disponibile per l'uso.

![Fine-tuning job status](../../../../../translated_images/it/fine-tuned-model-status.563271727bf7bfba.png)


Puoi anche visualizzare i messaggi di stato e le metriche scorrendo ulteriormente nella dashboard visiva come mostrato:

| Messages | Metrics |
|:---|:---|
| ![Messages](../../../../../translated_images/it/fine-tuned-messages-panel.4ed0c2da5ea1313b.png) |  ![Metrics](../../../../../translated_images/it/fine-tuned-metrics-panel.700d7e4995a65229.png)|


---

### Passo 3.1: Recupera l'ID e testa il modello fine-tuned nel codice


In [46]:
# Retrieve the identity of the fine-tuned model once ready
response = client.fine_tuning.jobs.retrieve(ft_filejob.id)
fine_tuned_model_id = response.fine_tuned_model
print("Fine-tuned Model ID:", fine_tuned_model_id)

Fine-tuned Model ID: ft:gpt-3.5-turbo-0125:bitnbot::9OFWzNjz


In [47]:
# You can then use that model to generate completions from the SDK as shown
# Or you can load that model into the OpenAI Playground (in the UI) to validate it from there.
from openai import OpenAI
client = OpenAI()

completion = client.chat.completions.create(
  model=fine_tuned_model_id,
  messages=[
    {"role": "system", "content": "You are Elle, a factual chatbot that answers questions about elements in the periodic table with a limerick"},
    {"role": "user", "content": "Tell me about Strontium"},
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="Strontium, a metal so bright - It's in fireworks, a dazzling sight - It's in bones, you see - And in tea, it's the key - It's the fortieth, so pure, that's the right", role='assistant', function_call=None, tool_calls=None)


---

### Passo 3.2: Carica e testa il modello fine-tuned nel Playground

Ora puoi testare il modello fine-tuned in due modi. Prima, puoi visitare il Playground e usare il menu a tendina Models per selezionare il tuo modello fine-tuned tra le opzioni elencate. L'altra opzione è usare l'opzione "Playground" mostrata nel pannello Fine-tuning (vedi screenshot sopra) che avvia la seguente vista _comparativa_ che mostra le versioni del modello foundation e fine-tuned affiancate per una valutazione rapida.

![Fine-tuning job status](../../../../../translated_images/it/fine-tuned-playground-compare.56e06f0ad8922016.png)

Basta inserire il contesto di sistema usato nei tuoi dati di addestramento e fornire la tua domanda di test. Noterai che entrambi i lati vengono aggiornati con lo stesso contesto e domanda. Esegui il confronto e vedrai la differenza nelle risposte tra i due. _Nota come il modello fine-tuned rende la risposta nel formato che hai fornito nei tuoi esempi mentre il modello foundation segue semplicemente il prompt di sistema_.

![Fine-tuning job status](../../../../../translated_images/it/fine-tuned-playground-launch.5a26495c983c6350.png)

Noterai che il confronto fornisce anche il conteggio dei token per ciascun modello e il tempo impiegato per l'inferenza. **Questo esempio specifico è semplice e serve a mostrare il processo ma non riflette un dataset o scenario reale**. Potresti notare che entrambi i campioni mostrano lo stesso numero di token (contesto di sistema e prompt utente sono identici) con il modello fine-tuned che impiega più tempo per l'inferenza (modello personalizzato).

In scenari reali, non userai un esempio giocattolo come questo, ma farai fine-tuning su dati reali (ad esempio, catalogo prodotti per il servizio clienti) dove la qualità della risposta sarà molto più evidente. In _quel_ contesto, ottenere una qualità di risposta equivalente con il modello foundation richiederà più ingegneria del prompt personalizzata che aumenterà l'uso dei token e potenzialmente il tempo di elaborazione per l'inferenza. _Per provare, dai un'occhiata agli esempi di fine-tuning nell'OpenAI Cookbook per iniziare._

---


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Pur impegnandoci per garantire l’accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa deve essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un traduttore umano. Non ci assumiamo alcuna responsabilità per eventuali malintesi o interpretazioni errate derivanti dall’uso di questa traduzione.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
